In [3]:
import numpy as np
import pandas as pd

КАК НУЖНО СДЕДАТЬ СТРУКУТУРУ КЛАССА

Переделать в остальных

либо соблюдать такую структура в дальнейшем

In [5]:
class MyLineReg():
    def __init__(self, n_iter=100, learning_rate=0.1, reg=None, l1_coef=0, l2_coef=0, metric=None, weights=None):
        self.n_iter = n_iter
        self.learning_rate = learning_rate
        self.metric = metric
        self.weights = weights
        self.reg = reg
        self.l1_coef = l1_coef
        self.l2_coef = l2_coef
        self.best_score = None

        # Валидация входных параметров
        self.__validate_params()

    def __str__(self):
        return f'MyLineReg class: n_iter={self.n_iter}, learning_rate={self.learning_rate}'

    def __validate_params(self):
        """Проверка корректности переданных параметров."""
        if self.metric is not None and self.metric not in ['mae', 'mse', 'rmse', 'mape', 'r2']:
            raise ValueError("metric must be 'mae', 'mse', 'rmse', 'mape' or 'r2'")
        if self.reg is not None and self.reg not in ['l1', 'l2', 'elasticnet']:
            raise ValueError("reg must be 'l1', 'l2', or 'elasticnet'")
        if self.l1_coef < 0 or self.l1_coef > 1:
            raise ValueError("l1_coef must be in [0, 1]")
        if self.l2_coef < 0 or self.l2_coef > 1:
            raise ValueError("l2_coef must be in [0, 1]")

    def __calculate_metric(self, pred: np.array, y_true: np.array, metric: str) -> float:
        """Вычисляет метрику качества."""
        if metric == 'mae':
            return np.mean(np.abs(pred - y_true))
        elif metric == 'mse':
            return np.mean((pred - y_true) ** 2)
        elif metric == 'rmse':
            return np.sqrt(np.mean((pred - y_true) ** 2))
        elif metric == 'mape':
            return 100 * np.mean(np.abs((pred - y_true) / y_true))
        elif metric == 'r2':
            ss_res = np.sum((pred - y_true) ** 2)
            ss_tot = np.sum((y_true - np.mean(y_true)) ** 2)
            return 1 - (ss_res / ss_tot)
        else:
            raise ValueError(f"Unknown metric: {metric}")

    def __calculate_loss(self, X: np.array, y: np.array, w: np.array) -> float:
        """Вычисляет общий loss (MSE + регуляризация)."""
        error = X.dot(w) - y
        mse = np.mean(error ** 2)
        
        if self.reg == 'l1':
            return mse + self.l1_coef * np.sum(np.abs(w))
        elif self.reg == 'l2':
            return mse + self.l2_coef * np.sum(w ** 2)
        elif self.reg == 'elasticnet':
            return mse + self.l1_coef * np.sum(np.abs(w)) + self.l2_coef * np.sum(w ** 2)
        else:
            return mse

    def __apply_regularization(self, grad: np.array, w: np.array) -> np.array:
        """Добавляет градиент регуляризации."""
        if self.reg == 'l1':
            grad += self.l1_coef * np.sign(w)
        elif self.reg == 'l2':
            grad += 2 * self.l2_coef * w
        elif self.reg == 'elasticnet':
            grad += self.l1_coef * np.sign(w) + 2 * self.l2_coef * w
        return grad

    def __get_learning_rate(self, iteration: int) -> float:
        """Вычисляет learning_rate (статический или динамический)."""
        if callable(self.learning_rate):
            return self.learning_rate(iteration)
        return self.learning_rate

    def __log_training_step(self, iteration: int, X: np.array, y: np.array, w: np.array):
        """Логирует процесс обучения."""
        loss = self.__calculate_loss(X, y, w)
        if self.metric:
            metric_value = self.__calculate_metric(X.dot(w), y, self.metric)
            print(f'Iteration {iteration} | loss: {loss:.2f} | {self.metric}: {metric_value:.2f}')
        else:
            print(f'Iteration {iteration} | loss: {loss:.2f}')

    def fit(self, X: pd.DataFrame, y: pd.Series, verbose=False):
        """Обучение модели."""
        X_copy = X.copy()
        X_copy.insert(0, 'base', 1)
        w = np.ones(X_copy.shape[1])

        if verbose:
            initial_loss = self.__calculate_loss(X_copy, y, w)
            print(f'start | loss: {initial_loss:.2f}')

        for i in range(1, self.n_iter + 1):
            pred = X_copy.dot(w)
            error = pred - y
            grad = (2 / len(X_copy)) * error.T.dot(X_copy)
            grad = self.__apply_regularization(grad, w)

            if self.metric:
                self.best_score = self.__calculate_metric(pred, y, self.metric)

            if verbose and (i % 10 == 0 or i == self.n_iter):
                self.__log_training_step(i, X_copy, y, w)

            lr = self.__get_learning_rate(i)
            w -= lr * grad

        self.weights = w

    def predict(self, X: pd.DataFrame) -> np.array:
        """Предсказание."""
        X_copy = X.copy()
        X_copy.insert(0, 'base', 1)
        return X_copy.dot(self.weights)

    def get_best_score(self) -> float:
        """Возвращает лучшее значение метрики."""
        if self.metric is None:
            raise ValueError("Metric was not set during model initialization.")
        return self.best_score

    def get_coef(self) -> np.array:
        """Возвращает коэффициенты модели (без intercept)."""
        return self.weights[1:]